# Capstone Project - The Battle of Neighborhoods

# Brooklyn Food Truck Location Recommendations

# I. PURPOSE

This document provides the details of my peer reviewed assignment for the IBM Data Science Professional Certificate Capstone project.

# II. INTRODUCTION

New York City is the largest city in U.S.A by population, which also encompasses five divisions called boroughs.  Of the five boroughs we will focus on Brooklyn, which is known for its night life and great food.

# III. Objective

In this project, we will study the areas of Brooklyn in order to determine which neighborhoods have the highest concentration of bars in order to solve our business problem.  

## A description of the problem and discussion of the background. (15 marks)

Our business problem is that we are working with a Food Truck business owner who is trying to determine where he should take his food truck for the greatest chance of success.<br/>
The aim of this project is to segment area's of Brooklyn's neighborhoods based on the concentration of bars using the Foursquare data.  <br/>We therorize that the best places to take a food truck at night is near areas that have high concentrations of bars.<br/>  This theory is based on two main principals.<br/>
1. Not all bars have a kitchen and sell food, so targeting these areas fills a need for our customers.<br/>
2. People going home from the bars are more likely to buy food to enjoy late at night when many other food options are closed.<br/>

Thus the area's with the highest concentration of bars will have the highest concentrations of potential customers.<br/>

## A description of the data and how it will be used to solve the problem. (15 marks)

We will be using the Foursquare Places API for our data source. https://developer.foursquare.com/docs/api<br/>
From this API we will use the Venue Categories and Venue Location data to map our data on a Folium map.<br/>
From here we will use K-Means clustering to find the ideal locations to send the Food Trucks to.

# III. Methodology

In order to determine where is the best place to take our food truck we will take a multi step process.  
1. First we will determine which neighborhood in Brooklyn we should focus on.  
2. We will then target a specific area of this neighborhood with our Foursquare data in order to determine which area will have the highest number of potential customers for our food truck business.  

We will use K-Means clustering to group our neighborhoods by common venues, and from there we will use information from the Foursquare API such as venue rating to determine which bars are the most popular.

### Import necessary Libraries

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np # library to handle data in a vectorized manner
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         238 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.50-py_0        conda-forge
    geopy:         1.20.0-py_0      conda-forge

The following pac

<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Brooklyn</font></h1>

## 1. Download and Explore Dataset

Neighborhood has a total of 5 boroughs and 306 neighborhoods. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood. 

Luckily, this dataset exists for free on the web. Feel free to try to find this dataset on your own, but here is the link to the dataset: https://geo.nyu.edu/catalog/nyu_2451_34572

In [2]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


#### Load and explore the data

Next, let's load the data.

In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [4]:
# commenting out for now due to large size making it hard to read
#newyork_data

In [5]:
neighborhoods_data = newyork_data['features']

In [6]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

#### Tranform the data into a *pandas* dataframe

The next task is essentially transforming this data of nested Python dictionaries into a *pandas* dataframe. So let's start by creating an empty dataframe.

In [7]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

Take a look at the empty dataframe to confirm that the columns are as intended.

In [8]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


Then let's loop through the data and fill the dataframe one row at a time.

In [9]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

Quickly examine the resulting dataframe.

In [10]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


And make sure that the dataset has all 5 boroughs and 306 neighborhoods.

In [11]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


#### Use geopy library to get the latitude and longitude values of Brooklyn, NY.

In [12]:
address = 'Brooklyn, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Brooklyn are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Brooklyn are 40.6501038, -73.9495823.


#### Create a map of New York with neighborhoods superimposed on top.

In [13]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10,)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

**Folium** is a great visualization library. Feel free to zoom into the above map, and click on each circle mark to reveal the name of the neighborhood and its respective borough.

However, for illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods in Brooklyn. So let's slice the original dataframe and create a new dataframe of the Brooklyn data.

In [14]:
brooklyn_data = neighborhoods[neighborhoods['Borough'] == 'Brooklyn'].reset_index(drop=True)
brooklyn_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Brooklyn,Bay Ridge,40.625801,-74.030621
1,Brooklyn,Bensonhurst,40.611009,-73.995180
2,Brooklyn,Sunset Park,40.645103,-74.010316
3,Brooklyn,Greenpoint,40.730201,-73.954241
4,Brooklyn,Gravesend,40.595260,-73.973471


In [15]:
address = 'Brooklyn, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Brooklyn are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Brooklyn are 40.6501038, -73.9495823.


In [16]:
# create map of Manhattan using latitude and longitude values
map_brooklyn = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(brooklyn_data['Latitude'], brooklyn_data['Longitude'], brooklyn_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_brooklyn)  
    
map_brooklyn

## 2. Explore Neighborhoods in Brooklyn

#### Let's create a function to repeat the same process to all the neighborhoods in Brooklyn

In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *brooklyn_venues*.

#### Define Foursquare Credentials and Version

In [18]:
CLIENT_ID = 'CVFZIHHKFIYX5VCCCJJLKGBEKGTCXA2VLQOCMFTXJNFRMIS2' # your Foursquare ID
CLIENT_SECRET = 'IHUGWTRHYEUYIZOP1UAMJKI1FAWCPHI5H22SAHAR1WGF1TXZ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 1000 # define radius

Your credentails:
CLIENT_ID: CVFZIHHKFIYX5VCCCJJLKGBEKGTCXA2VLQOCMFTXJNFRMIS2
CLIENT_SECRET:IHUGWTRHYEUYIZOP1UAMJKI1FAWCPHI5H22SAHAR1WGF1TXZ


In [19]:
brooklyn_venues = getNearbyVenues(names=brooklyn_data['Neighborhood'],
                                   latitudes=brooklyn_data['Latitude'],
                                   longitudes=brooklyn_data['Longitude']
                                  )

Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker Heights
Gerritsen Beach
Marine Park
Clinton Hill
Sea Gate
Downtown
Boerum Hill
Prospect Lefferts Gardens
Ocean Hill
City Line
Bergen Beach
Midwood
Prospect Park South
Georgetown
East Williamsburg
North Side
South Side
Ocean Parkway
Fort Hamilton
Ditmas Park
Wingate
Rugby
Remsen Village
New Lots
Paerdegat Basin
Mill Basin
Fulton Ferry
Vinegar Hill
Weeksville
Broadway Junction
Dumbo
Homecrest
Highland Park
Madison
Erasmus


#### Let's check the size of the resulting dataframe

In [20]:
print(brooklyn_venues.shape)
brooklyn_venues.head()

(2845, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bay Ridge,40.625801,-74.030621,Pilo Arts Day Spa and Salon,40.624748,-74.030591,Spa
1,Bay Ridge,40.625801,-74.030621,Bagel Boy,40.627896,-74.029335,Bagel Shop
2,Bay Ridge,40.625801,-74.030621,Cocoa Grinder,40.623967,-74.030863,Juice Bar
3,Bay Ridge,40.625801,-74.030621,Pegasus Cafe,40.623168,-74.031186,Breakfast Spot
4,Bay Ridge,40.625801,-74.030621,Ho' Brah Taco Joint,40.622960,-74.031371,Taco Place


Let's check how many venues were returned for each neighborhood

In [21]:
brooklyn_venues.groupby('Neighborhood').count().head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bath Beach,49,49,49,49,49,49
Bay Ridge,86,86,86,86,86,86
Bedford Stuyvesant,28,28,28,28,28,28
Bensonhurst,31,31,31,31,31,31
Bergen Beach,7,7,7,7,7,7


#### Let's find out how many unique categories can be curated from all the returned venues

In [22]:
print('There are {} uniques categories.'.format(len(brooklyn_venues['Venue Category'].unique())))

There are 286 uniques categories.


## 3. Analyze Each Neighborhood

In [23]:
# one hot encoding
brooklyn_onehot = pd.get_dummies(brooklyn_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
brooklyn_onehot['Neighborhood'] = brooklyn_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [brooklyn_onehot.columns[-1]] + list(brooklyn_onehot.columns[:-1])
brooklyn_onehot = brooklyn_onehot[fixed_columns]

brooklyn_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Herbs & Spices Store,History Museum,Hobby Shop,Hockey Field,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Non-Profit,Noodle House,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pie Shop,Piercing Parlor,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Road,Rock Club,Roller Rink,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Street Fair,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Tiki Bar,Toy / Game Store,Trail,Turkish Restaurant,Used Bookstore,Vape Stor

And let's examine the new dataframe size

In [24]:
brooklyn_onehot.shape

(2845, 286)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [25]:
brooklyn_grouped = brooklyn_onehot.groupby('Neighborhood').mean().reset_index()
brooklyn_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Herbs & Spices Store,History Museum,Hobby Shop,Hockey Field,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Non-Profit,Noodle House,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pie Shop,Piercing Parlor,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Road,Rock Club,Roller Rink,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Street Fair,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Tiki Bar,Toy / Game Store,Trail,Turkish Restaurant,Used Bookstore,Vape Stor

#### Let's confirm the new size

In [26]:
brooklyn_grouped.shape

(70, 286)

#### Let's print each neighborhood along with the top 5 most common venues

In [27]:
num_top_venues = 5

for hood in brooklyn_grouped['Neighborhood']:
    #print("----"+hood+"----")
    temp = brooklyn_grouped[brooklyn_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    #print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    #print('\n')

#### Let's put this into a pandas dataframe
First, lets write a function to sort the venues in descending order

In [28]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [29]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = brooklyn_grouped['Neighborhood']

for ind in np.arange(brooklyn_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(brooklyn_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bath Beach,Pharmacy,Fast Food Restaurant,Donut Shop,Italian Restaurant,Kids Store,Sushi Restaurant,Chinese Restaurant,Bubble Tea Shop,Pizza Place,German Restaurant
1,Bay Ridge,Spa,Italian Restaurant,Bar,Greek Restaurant,American Restaurant,Chinese Restaurant,Pizza Place,Bagel Shop,Hookah Bar,Sandwich Place
2,Bedford Stuyvesant,Café,Deli / Bodega,Pizza Place,Coffee Shop,Bar,Fried Chicken Joint,BBQ Joint,Fruit & Vegetable Store,Boutique,New American Restaurant
3,Bensonhurst,Ice Cream Shop,Sushi Restaurant,Donut Shop,Pizza Place,Grocery Store,Coffee Shop,Factory,Chinese Restaurant,Supermarket,Road
4,Bergen Beach,Harbor / Marina,Donut Shop,Athletics & Sports,Playground,Baseball Field,Hockey Field,Farm,Farmers Market,Fast Food Restaurant,Field


## 4. Cluster Neighborhoods
Run *k*-means to cluster the neighborhood into 5 clusters.

In [30]:
# set number of clusters
kclusters = 5

brooklyn_grouped_clustering = brooklyn_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(brooklyn_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

#### Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [31]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

brooklyn_merged = brooklyn_data

# merge brooklyn_grouped with brooklyn_data to add latitude/longitude for each neighborhood
brooklyn_merged = brooklyn_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

brooklyn_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Brooklyn,Bay Ridge,40.625801,-74.030621,0,Spa,Italian Restaurant,Bar,Greek Restaurant,American Restaurant,Chinese Restaurant,Pizza Place,Bagel Shop,Hookah Bar,Sandwich Place
1,Brooklyn,Bensonhurst,40.611009,-73.995180,0,Ice Cream Shop,Sushi Restaurant,Donut Shop,Pizza Place,Grocery Store,Coffee Shop,Factory,Chinese Restaurant,Supermarket,Road
2,Brooklyn,Sunset Park,40.645103,-74.010316,0,Latin American Restaurant,Bakery,Bank,Pizza Place,Mexican Restaurant,Gym,Deli / Bodega,Pharmacy,Mobile Phone Shop,Sandwich Place
3,Brooklyn,Greenpoint,40.730201,-73.954241,0,Bar,Cocktail Bar,Coffee Shop,Pizza Place,French Restaurant,Mexican Restaurant,Café,Yoga Studio,Record Shop,Sushi Restaurant
4,Brooklyn,Gravesend,40.595260,-73.973471,0,Pizza Place,Bus Station,Chinese Restaurant,Lounge,Gym,Italian Restaurant,Bakery,Breakfast Spot,Music Store,Electronics Store


#### Lets see which Neighborhoods have Bar as their most common venue.  We will use this later to decide which Neighborhoods to focus on.

In [32]:
bar_1st_most_common = brooklyn_merged[brooklyn_merged['1st Most Common Venue']=='Bar']
bar_1st_most_common['Neighborhood']

3            Greenpoint
13     Prospect Heights
16             Bushwick
49    East Williamsburg
51           South Side
Name: Neighborhood, dtype: object

In [33]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(brooklyn_merged['Latitude'], brooklyn_merged['Longitude'], brooklyn_merged['Neighborhood'], brooklyn_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

## Map of all clusters in Brooklyn

In [34]:
map_clusters

#### Lets recenter our map around Williamsburg

In [35]:
address = 'Williamsburg, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Williamsburg are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Williamsburg are 40.714622, -73.95345.


In [36]:
# create map
map_clusters_bars = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bar_1st_most_common['Latitude'], bar_1st_most_common['Longitude'], bar_1st_most_common['Neighborhood'], bar_1st_most_common['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters_bars)

## Map of clusters in Brooklyn where Bars are the most common venue

In [37]:
map_clusters_bars

## According to the data we should focus on venues around Williamsburg because it has the highest density of bars

# Next we will determine which area in Williamsburg is best to send our food truck

## 1. Search for a specific venue category
> `https://api.foursquare.com/v2/venues/`**search**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**`&query=`**QUERY**`&radius=`**RADIUS**`&limit=`**LIMIT**

In [38]:
search_query = 'Bar'
radius = 500
print(search_query + ' .... OK!')

Bar .... OK!


#### Define the corresponding URL

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Let's explore the first neighborhood in our dataframe.
Get the neighborhood's name.

In [39]:
brooklyn_data.loc[15, 'Neighborhood']

'Williamsburg'

|Get the neighborhood's latitude and longitude values.

In [40]:
neighborhood_latitude = brooklyn_data.loc[15, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = brooklyn_data.loc[15, 'Longitude'] # neighborhood longitude value

neighborhood_name = brooklyn_data.loc[15, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Williamsburg are 40.70714439344251, -73.95811529220927.


## Define the corresponding URL

In [41]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=CVFZIHHKFIYX5VCCCJJLKGBEKGTCXA2VLQOCMFTXJNFRMIS2&client_secret=IHUGWTRHYEUYIZOP1UAMJKI1FAWCPHI5H22SAHAR1WGF1TXZ&ll=40.714622,-73.95345&v=20180605&query=Bar&radius=500&limit=100'

#### Send the GET Request

In [42]:
results = requests.get(url).json()
#results

#### Get relevant part of JSON and transform it into a pandas dataframe


In [43]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name,venuePage.id,location.neighborhood
0,49bb36fcf964a520dc531fe3,Caracas Arepa Bar,"[{'id': '4bf58dd8d48988d152941735', 'name': 'A...",v-1571250021,False,291 Grand St,btwn Roebling & Havemeyer St,40.713075,-73.957462,"[{'label': 'display', 'lat': 40.71307533802476...",379,11211,US,Brooklyn,NY,United States,"[291 Grand St (btwn Roebling & Havemeyer St), ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4af33812f964a520d2eb21e3,Full Circle Bar,"[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",v-1571250021,False,318 Grand St,btwn Havemeyer St. & Marcy Ave.,40.712662,-73.956688,"[{'label': 'display', 'lat': 40.71266183593462...",349,11211,US,Brooklyn,NY,United States,[318 Grand St (btwn Havemeyer St. & Marcy Ave....,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4581734ff964a520653f1fe3,Surf Bar,"[{'id': '4bf58dd8d48988d1ce941735', 'name': 'S...",v-1571250021,False,139 N 6th St,btwn Bedford & Berry St.,40.717636,-73.958714,"[{'label': 'display', 'lat': 40.71763628703034...",556,11249,US,Brooklyn,NY,United States,"[139 N 6th St (btwn Bedford & Berry St.), Broo...",300434,https://www.seamless.com/menu/surf-bar-139-n-6...,seamless,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_seamless_20180129.png,78812387,NaN
3,4262f880f964a52025211fe3,Teddy's Bar & Grill,"[{'id': '4bf58dd8d48988d11b941735', 'name': 'P...",v-1571250021,False,96 Berry St,at N. 8th,40.719205,-73.958431,"[{'label': 'display', 'lat': 40.71920479788144...",660,11249,US,Brooklyn,NY,United States,"[96 Berry St (at N. 8th), Brooklyn, NY 11249, ...",316509,https://www.seamless.com/menu/teddys-bar-and-g...,seamless,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_seamless_20180129.png,78597371,NaN
4,54544a09498ea8054f640543,Dardy Bar,"[{'id': '4bf58dd8d48988d118941735', 'name': 'D...",v-1571250021,False,245 S. 1st Street,Roebling,40.712627,-73.957970,"[{'label': 'display', 'lat': 40.712627, 'lng':...",441,11211,US,Brooklyn,NY,United States,"[245 S. 1st Street (Roebling), Brooklyn, NY 11...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Define information of interest and filter dataframe

In [44]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Caracas Arepa Bar,Arepa Restaurant,291 Grand St,btwn Roebling & Havemeyer St,40.713075,-73.957462,"[{'label': 'display', 'lat': 40.71307533802476...",379,11211,US,Brooklyn,NY,United States,"[291 Grand St (btwn Roebling & Havemeyer St), ...",NaN,49bb36fcf964a520dc531fe3
1,Full Circle Bar,Bar,318 Grand St,btwn Havemeyer St. & Marcy Ave.,40.712662,-73.956688,"[{'label': 'display', 'lat': 40.71266183593462...",349,11211,US,Brooklyn,NY,United States,[318 Grand St (btwn Havemeyer St. & Marcy Ave....,NaN,4af33812f964a520d2eb21e3
2,Surf Bar,Seafood Restaurant,139 N 6th St,btwn Bedford & Berry St.,40.717636,-73.958714,"[{'label': 'display', 'lat': 40.71763628703034...",556,11249,US,Brooklyn,NY,United States,"[139 N 6th St (btwn Bedford & Berry St.), Broo...",NaN,4581734ff964a520653f1fe3
3,Teddy's Bar & Grill,Pub,96 Berry St,at N. 8th,40.719205,-73.958431,"[{'label': 'display', 'lat': 40.71920479788144...",660,11249,US,Brooklyn,NY,United States,"[96 Berry St (at N. 8th), Brooklyn, NY 11249, ...",NaN,4262f880f964a52025211fe3
4,Dardy Bar,Dive Bar,245 S. 1st Street,Roebling,40.712627,-73.957970,"[{'label': 'display', 'lat': 40.712627, 'lng':...",441,11211,US,Brooklyn,NY,United States,"[245 S. 1st Street (Roebling), Brooklyn, NY 11...",NaN,54544a09498ea8054f640543


#### Let's visualize the Bars that are nearby

In [45]:
dataframe_filtered.name

0                                     Caracas Arepa Bar
1                                       Full Circle Bar
2                                              Surf Bar
3                                   Teddy's Bar & Grill
4                                             Dardy Bar
5                            The West—Coffeehouse & Bar
6                                       Huckleberry Bar
7                                     Momofuku Milk Bar
8                                               Barcade
9                                       Desy's Clam Bar
10                                   Pinkerton Wine Bar
11                                          The Bar 245
12    Sister Margaret’s School For Wayward Girls - D...
13                                            Baker Bar
14                                             Bar Beau
15                                          North 4 Bar
16                                     Woodhul Wine Bar
17                                              

In [46]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=16) # generate map centred around Williamsburg Brooklyn

# add a red circle marker to represent Williamsburg Brooklyn
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Williamsburg',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Bars as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

## There appears to be a nice cluster of bars around Grand Street and Havemeyer Street.
## Lets investigate Caracas Arepa Bar since it is near that area

In [47]:
venue_id = '49bb36fcf964a520dc531fe3' # ID of Caracas Arepa Bar
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)


result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

9.1


## This is a great rating!

# Explore a location

## Let's explore the popular spots around the bar. In order to explore the area, let's start by getting the latitude and longitude values of Caracas Arepa Barz.

In [48]:
latitude = 40.71307533802476
longitude = -73.9574621701832

In [49]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=CVFZIHHKFIYX5VCCCJJLKGBEKGTCXA2VLQOCMFTXJNFRMIS2&client_secret=IHUGWTRHYEUYIZOP1UAMJKI1FAWCPHI5H22SAHAR1WGF1TXZ&ll=40.71307533802476,-73.9574621701832&v=20180605&radius=500&limit=100'

### Send GET request and examine results

In [50]:
results = requests.get(url).json()
'There are {} around Caracas Arepa Bar.'.format(len(results['response']['groups'][0]['items']))

'There are 100 around Caracas Arepa Bar.'

## Get the relevant part of the JSON data

In [51]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4e4ea8b5aeb70f12849528ed',
  'name': 'St. Mazie',
  'location': {'address': '345 Grand St',
   'crossStreet': 'btwn Marcy Ave & Havemeyer St',
   'lat': 40.712446182313414,
   'lng': -73.95595408442468,
   'labeledLatLngs': [{'label': 'display',
     'lat': 40.712446182313414,
     'lng': -73.95595408442468}],
   'distance': 145,
   'postalCode': '11211',
   'cc': 'US',
   'city': 'Brooklyn',
   'state': 'NY',
   'country': 'United States',
   'formattedAddress': ['345 Grand St (btwn Marcy Ave & Havemeyer St)',
    'Brooklyn, NY 11211',
    'United States']},
  'categories': [{'id': '4bf58dd8d48988d11e941735',
    'name': 'Cocktail Bar',
    'pluralName': 'Cocktail Bars',
    'shortName': 'Cocktail',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/cocktails_',
     'suffix': '.png'},
    'primary': Tru

## Process JSON and convert it to a clean dataframe

In [52]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,St. Mazie,Cocktail Bar,345 Grand St,btwn Marcy Ave & Havemeyer St,40.712446,-73.955954,"[{'label': 'display', 'lat': 40.71244618231341...",145,11211,US,Brooklyn,NY,United States,"[345 Grand St (btwn Marcy Ave & Havemeyer St),...",NaN,4e4ea8b5aeb70f12849528ed
1,East River Pilates,Pilates Studio,235 S 1st St,at Roebling street,40.712891,-73.958500,"[{'label': 'display', 'lat': 40.71289117042526...",89,11211,US,Brooklyn,NY,United States,"[235 S 1st St (at Roebling street), Brooklyn, ...",NaN,54df7b38498eee7bd051f0bc
2,L'Industrie Pizzeria,Pizza Place,254 S 2nd St,Havemeyer,40.711675,-73.957697,"[{'label': 'display', 'lat': 40.7116752206617,...",157,11211,US,Brooklyn,NY,United States,"[254 S 2nd St (Havemeyer), Brooklyn, NY 11211,...",NaN,55135426498e9dee8f19cc5f
3,St. Anselm,American Restaurant,355 Metropolitan Ave,btwn Havemeyer & Roebling St,40.714145,-73.955984,"[{'label': 'display', 'lat': 40.71414471294114...",172,11211,US,Brooklyn,NY,United States,[355 Metropolitan Ave (btwn Havemeyer & Roebli...,NaN,4be5bbe8d4f7c9b6b9d52520
4,Emmy Squared,Pizza Place,364 Grand St,at Marcy Ave,40.712166,-73.955705,"[{'label': 'display', 'lat': 40.712166, 'lng':...",179,11211,US,Brooklyn,NY,United States,"[364 Grand St (at Marcy Ave), Brooklyn, NY 112...",NaN,570139a9498e16178005c729
5,Twenty Sided Store,Toy / Game Store,362 Grand St,at Marcy St,40.712268,-73.955476,"[{'label': 'display', 'lat': 40.71226832925318...",190,11211,US,Brooklyn,NY,United States,"[362 Grand St (at Marcy St), Brooklyn, NY 1121...",NaN,4d9764eb942ba093f86e6b8c
6,Have & Meyer Chatteria,Wine Bar,103 Havermeyer St,Hope St,40.713305,-73.956464,"[{'label': 'display', 'lat': 40.71330486799995...",88,11211,US,Brooklyn,NY,United States,"[103 Havermeyer St (Hope St), Brooklyn, NY 112...",NaN,563169d2498e609fa36f7bd3
7,The Four Horsemen,Wine Bar,295 Grand St,btwn Havemeyer & Roebling St,40.712971,-73.957349,"[{'label': 'display', 'lat': 40.71297062908573...",15,11211,US,Brooklyn,NY,United States,"[295 Grand St (btwn Havemeyer & Roebling St), ...",NaN,554f66e8498e387866311421
8,Overthrow Boxing,Boxing Gym,256 Grand St,NaN,40.713419,-73.958759,"[{'label': 'display', 'lat': 40.71341935159598...",115,11211,US,New York,NY,United States,"[256 Grand St, New York, NY 11211, United States]",NaN,59a013947564f768b785adfb
9,Bozu,Sushi Restaurant,296 Grand St,btwn Roebling & Havemeyer St,40.712976,-73.957478,"[{'label': 'display', 'lat': 40.71297626571757...",11,11211,US,Brooklyn,NY,United States,"[296 Grand St (btwn Roebling & Havemeyer St), ...",NaN,423e0e80f964a52048201fe3


## Let's visualize these venues on the map around our location

In [53]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=17) # generate map centred around Caracas Arepa Bar


# add Ecco as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Caracas Arepa Bar',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)

# add the Bars as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        #popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)


# display map
venues_map

## This shows there is a high density of venues and people near this area

# Results Section

#### We followed the methodology that way described above and came to the conclusion that Williamsburg was the best place to send our food truck and of the areas in Williamsburg to focus on, the area near Grand Street and Havemeyer Street would give us the best location to send our food truck

# Discussion Section

#### Observations that I made during this analysis were that Brooklyn has many areas and neighborhoods where bars can be found.  For our purposes we wanted to find the area that would have the highest concentration of bars and night life.  We came to the conclusion that Williamsburg had the highest concentration of bars within a walking distance of each other.  

#### Knowing that people in Brooklyn tend to walk to most places we want to send our food truck to places that will have high foot traffic.  We were able to isolate a specific corner in Williamsburg (Grand Street and Havemeyer Street) that would be a good location for our food truck.  This is because there are many venues around this location that would have potential customers.  

#### Further considerations for improving this model would be the time of day, which has an impact on foot traffic in Brooklyn.  Also, we might also want to consider how far away mass transit is from our location.  We might be able to get high foot traffic by finding locations that are near subway stops in Williamsburg as well.  

# Conclusion Section

### In conclusion, we have shown that k-means clustering combined with Foursquare API data can be used to make business recommendations leveraging machine learning methods.  We were able to find a specific area in a specific neighborhood that optimizes our chances for success.